In [2]:
import pandas as pd
import json
from nostr_sdk import Keys, Client, EventBuilder, Filter, PublicKey
from datetime import timedelta
import time

In [5]:
keys = Keys.generate()
print(keys.public_key().to_bech32())
client = Client(keys)
client.add_relay("wss://relay.damus.io")
client.connect()
# PublicKey.from_hex('031ea306188fee631a86f1a0e49cee8f3d62592b2e7be5f8eb1788e327db340a').to_bech32()

npub1v9wndp52p34hqcvdqkdgw0ut87hfxsva6gj9tfh0kylygc4juyeszc3cht


In [6]:
type(keys), type(keys.public_key()), type(keys.public_key().to_bech32())

(nostr_sdk.nostr_sdk.Keys, nostr_sdk.nostr_sdk.PublicKey, str)

In [7]:
def query_events(pubkey):
    pk = PublicKey.from_bech32(pubkey)
    filter = Filter().kind(1).author(pk.to_hex()).limit(10)
    events = client.get_events_of([filter], timedelta(seconds=10))
    return pd.DataFrame([json.loads(event.as_json()) for event in events])
    

In [8]:
dfd = query_events("npub1dergggklka99wwrs92yz8wdjs952h2ux2ha2ed598ngwu9w7a6fsh9xzpc")

In [11]:
filter = Filter().kind(1).limit(100)
events = client.get_events_of([filter], timedelta(seconds=10))
df = pd.DataFrame([json.loads(event.as_json()) for event in events])

In [14]:
df.to_csv('events_100.csv')

In [ ]:
# convert hex to bech32: 031ea306188fee631a86f1a0e49cee8f3d62592b2e7be5f8eb1788e327db340a

In [15]:
df.loc[0, 'content']

'まだやってるのか・・・＞【声出し復活！】チルノのパーフェクトさんすう教室踊ってみたオフ2023 in 北海道 【バーカ！バーカ！】#チルノオフ\nhttps://www.nicovideo.jp/watch/sm41623576?ref=nostr\n#sm41623576\n#ニコニコ動画'

In [17]:
df.to_pickle('events_100.pkl')

In [18]:
df2 = pd.read_pickle('events_100.pkl')

In [19]:
df2

,content,created_at,id,kind,pubkey,sig,tags
0,まだやってるのか・・・＞【声出し復活！】チルノのパーフェクトさんすう教室踊ってみたオフ202...,1694435660,aa140b39d834edf433980e6fe7d0f6a011e4ee0ba435f8...,1,1f617e368ce633acef348a2f755dd0a459e56e39476669...,51c2b9fb88895aab8cecf762cafaf8b921ec92bbf2eb6d...,"[[t, sm41623576], [client, coracle]]"
1,What is the last thing to go through a fly's m...,1694435658,6fd2ab59dfd3a0979581ec04969069221ae0491cc22570...,1,d690ba058d9c4f6f4fc91438289817df19da4e04494dc4...,38a2bb0645731613941d388c304c31ae96f6939abf190b...,[]
2,https://image.nostr.build/f432803781d6004bed1a...,1694435656,232ee4cfbdd378cc20c6ab0e7bb7e4912c8344d0d8bba8...,1,78e47dfe683faf1869a5378aa4b4a52d768d799bd8eb38...,a07421297ab12e1e51f21fec894b7b16becf857a9f8b6f...,[]
3,Most great people have attained their greatest...,1694435652,c0889790ead173575e57fdf7adbb053093d1f7b4d72c38...,1,1987f7a6dab8428d8fd14721fbaf1ff28a2c61817f86dc...,4d897f9cb7541714be1b912707c789b946a673de402ae9...,[]
4,wow makna sat ternyata dalam juga ya. cocok ba...,1694435650,d277c1c96983b576eda19718b8855d26e20d223d9ff6e4...,1,c3495ddc53079b931f8727e7f7428c93f1612266210c0a...,af3e10de065b4e324d0cd9c266b89f88a260648d98ed79...,[]
...,...,...,...,...,...,...,...
95,https://youtu.be/--bIyenPc20?si=rOBaQ3hhSm-Svx...,1694435410,46493cc0e9c8137599f41fb600a7b588fddb72b8881e50...,1,cd68b38a24db7f395d26fa1a053b5bffdfe35f7811af53...,ad583e0653836a97a8f5bb1f902d922d9ab9fecc4bc697...,"[[t, 911]]"
96,風呂った,1694435409,f550e8635da04c6c98e26db72971d1189ff0af17cb7f33...,1,0c9b1e9fef76c88b63f86645dc33bb7777f0259ec41e67...,93abb20e2c40fff22542321f3ddd6153ddcbe2bc848079...,[]
97,Gm and pv! Have a great day fren ☯️💜⚡🫂,1694435409,b152009f4ebc328b9cfb3a2a2859ae04a38e762d5efde2...,1,f839fb6714598a7233d09dbd42af82cc9781d0faa57474...,96cd0d7d3ff218e47b9ce6d613adeea63e924a37abc7b0...,"[[e, f4aa3e803d1069a54bea8d9f36f44be72d9340c42..."
98,Fancy.,1694435408,968a5d10c7283c08bd4d7a526325f6cbf93e1f3e79c54d...,1,717d4b62ed661dec51fb39962728a424dd00f8edb68da2...,642de29575434285741163a98de7b899f8d8dd6b8317ab...,"[[e, 0f9390a6cc2fcac800fa253964ca151576ae4557a..."


In [55]:
pk.to_hex(), pk.to_nostr_uri()

('787338757fc25d65cd929394d5e7713cf43638e8d259e8dcf5c73b834eb851f2',
 'nostr:npub10pensatlcfwktnvjjw2dtem38n6rvw8g6fv73h84cuacxn4c28eqyfn34f')

In [47]:
# pk = PublicKey.from_bech32("npub1dergggklka99wwrs92yz8wdjs952h2ux2ha2ed598ngwu9w7a6fsh9xzpc")
pk = PublicKey.from_bech32("npub10pensatlcfwktnvjjw2dtem38n6rvw8g6fv73h84cuacxn4c28eqyfn34f")

# print(f"\nGetting profile metadata for {pk.to_bech32()}...")
# filter = Filter().kind(0).author(pk.to_hex()).limit(1)
# events = client.get_events_of([filter], timedelta(seconds=10))
# for event in events:
#     metadata = Metadata.from_json(event.content())
#     print(f"Name: {metadata.get_name()}")
#     print(f"NIP05: {metadata.get_nip05()}")
#     print(f"LUD16: {metadata.get_lud16()}")

filter = Filter().kind(1).author(pk.to_hex()).limit(10)
events = client.get_events_of([filter], timedelta(seconds=10))
for event in events:
    # print(json.dumps(event.as_json, indent=4))
    print(event.as_json())
    

{"content":"Let's make things pretty. And useful. And awesome. #nostrdesign \n\nhttps://nostrdesign.org/","created_at":1694211711,"id":"f9ce617c89cbe10168a11d67ec7cc4695ec35592cee0d4f7ac8267dd09543dad","kind":1,"pubkey":"787338757fc25d65cd929394d5e7713cf43638e8d259e8dcf5c73b834eb851f2","sig":"c2c4a6f76aab7f25236acffabd15e9c9cd9874d1f410355ab7bca254086bd3a838fe68cf6e48870e993e5a69eef20a4d93d9dd6e08af45f29c0ac7aa141ddc7a","tags":[["t","nostrdesign"],["t","nostrdesign"],["r","https://nostrdesign.org/"]]}
{"content":"Thank you nostr:npub1r0rs5q2gk0e3dk3nlc7gnu378ec6cnlenqp8a3cjhyzu6f8k5sgs4sq9ac and nostr:npub10000003zmk89narqpczy4ff6rnuht2wu05na7kpnh3mak7z2tqzsv8vwqk for spearheading this initiative 🙏💜","created_at":1694211554,"id":"e68984b1d8f393595fc5ed7b36d1bc4700c9b6bacbae7d5f5c43c4a455b7bedb","kind":1,"pubkey":"787338757fc25d65cd929394d5e7713cf43638e8d259e8dcf5c73b834eb851f2","sig":"c3b8cd38acecd6c376796bcb87709b1998c38e14109a6c7434f6930042bf8e9712a5fdcff36b46a0af9c61e5cd0ca3921f44ab

In [26]:
for event in events:
    print(json.dumps(event.as_json(), indent=4))
    # print(event.as_json())
    

"{\"content\":\"Let's make things pretty. And useful. And awesome. #nostrdesign \\n\\nhttps://nostrdesign.org/\",\"created_at\":1694211711,\"id\":\"f9ce617c89cbe10168a11d67ec7cc4695ec35592cee0d4f7ac8267dd09543dad\",\"kind\":1,\"pubkey\":\"787338757fc25d65cd929394d5e7713cf43638e8d259e8dcf5c73b834eb851f2\",\"sig\":\"c2c4a6f76aab7f25236acffabd15e9c9cd9874d1f410355ab7bca254086bd3a838fe68cf6e48870e993e5a69eef20a4d93d9dd6e08af45f29c0ac7aa141ddc7a\",\"tags\":[[\"t\",\"nostrdesign\"],[\"t\",\"nostrdesign\"],[\"r\",\"https://nostrdesign.org/\"]]}"
"{\"content\":\"Thank you nostr:npub1r0rs5q2gk0e3dk3nlc7gnu378ec6cnlenqp8a3cjhyzu6f8k5sgs4sq9ac and nostr:npub10000003zmk89narqpczy4ff6rnuht2wu05na7kpnh3mak7z2tqzsv8vwqk for spearheading this initiative \ud83d\ude4f\ud83d\udc9c\",\"created_at\":1694211554,\"id\":\"e68984b1d8f393595fc5ed7b36d1bc4700c9b6bacbae7d5f5c43c4a455b7bedb\",\"kind\":1,\"pubkey\":\"787338757fc25d65cd929394d5e7713cf43638e8d259e8dcf5c73b834eb851f2\",\"sig\":\"c3b8cd38acecd6c376796b

In [32]:
display(JSON(event.as_json()))

<IPython.core.display.JSON object>

In [51]:
df = pd.DataFrame(map(lambda x: json.loads(x.as_json()), events))

In [52]:
type(df.iloc[0])

pandas.core.series.Series

In [1]:
df.iloc[0]

NameError: name 'df' is not defined

In [54]:
df

,content,created_at,id,kind,pubkey,sig,tags
0,Let's make things pretty. And useful. And awes...,1694211711,f9ce617c89cbe10168a11d67ec7cc4695ec35592cee0d4...,1,787338757fc25d65cd929394d5e7713cf43638e8d259e8...,c2c4a6f76aab7f25236acffabd15e9c9cd9874d1f41035...,"[[t, nostrdesign], [t, nostrdesign], [r, https..."
1,Thank you nostr:npub1r0rs5q2gk0e3dk3nlc7gnu378...,1694211554,e68984b1d8f393595fc5ed7b36d1bc4700c9b6bacbae7d...,1,787338757fc25d65cd929394d5e7713cf43638e8d259e8...,c3b8cd38acecd6c376796bcb87709b1998c38e14109a6c...,"[[e, 3c1957931ef93baba7dccbdb3e32679ca21fdf035..."
2,Are you working on an open-source project that...,1694211510,3c1957931ef93baba7dccbdb3e32679ca21fdf035f7ebb...,1,787338757fc25d65cd929394d5e7713cf43638e8d259e8...,7eb77f8a015004bf4600c5d35933f1b0d4c876d543f914...,"[[t, nostrdesign], [t, nostrdesign], [r, desig..."
3,"New month, new LTS grantee! nostr:npub1s6z7hmm...",1694166940,3ff4d33e1016b1e49a59d4e3dbdd43f36b05bf82d501a5...,1,787338757fc25d65cd929394d5e7713cf43638e8d259e8...,800d5c392bd4dead93205bb8c6a7de92a1e5bd938b26d1...,"[[p, 8685ebef665338dd6931e2ccdf3c19d9f0e5a1067..."
4,👀,1694037444,cc904fefbdf3a8954d8881b3c95db2959940bfe5528f64...,1,787338757fc25d65cd929394d5e7713cf43638e8d259e8...,d9aa3f2dfc8fe5e7f072eccf09566e9664eaa75456093a...,"[[e, 202f72d1d99fa82ff1660011a22f269b6ba6d8758..."
5,Know someone who works on open-source freedom ...,1693983764,d66b161bc6e0ec759693a5e7c5568e58941618e6ae54d3...,1,787338757fc25d65cd929394d5e7713cf43638e8d259e8...,882b415276a247139320bdbf6893731936b20c1d707bb9...,"[[r, https://opensats.org/apply]]"
6,We're hiring! #jobstr \n\nhttps://bitcoinerjob...,1693385269,8f88a5f55452b94da4e282b2a3cfe9d96606de527be4e0...,1,787338757fc25d65cd929394d5e7713cf43638e8d259e8...,467a17e19f8ce102d484ac87c6018f10dfde98b5adebbc...,"[[t, jobstr], [t, jobstr], [r, https://bitcoin..."
7,"Can't stop, won't stop. OpenSats is providing ...",1692282438,a262e5b2b55f285d91e825a0f8751bf324294112a74540...,1,787338757fc25d65cd929394d5e7713cf43638e8d259e8...,4225c964e90c6b4bb34c988bcd04223efcf993322f7323...,"[[t, bitcoin], [t, nostr]]"
8,"You build, we support 🤝\n\nnostr:nevent1qqs9sx...",1692267052,4a7f963891d68802d122cea71369b81ecb84c33f578e7a...,1,787338757fc25d65cd929394d5e7713cf43638e8d259e8...,cbc419a019ab11320e2af8880771f542c23852c8aab3f2...,"[[e, 581a3a43b2542ed73a8ab3897d9951d8ffb7515ef..."
9,"Thank you for building, Josi. We're happy to s...",1692219512,bb11666d51331ed440b8f573de42b798e0de67a1cea7d5...,1,787338757fc25d65cd929394d5e7713cf43638e8d259e8...,bd74dfe3fdafad7205882c5cf9c61c68d0609ecdd21d23...,[]


In [46]:
events[0].as_json()

'{"content":"Let\'s make things pretty. And useful. And awesome. #nostrdesign \\n\\nhttps://nostrdesign.org/","created_at":1694211711,"id":"f9ce617c89cbe10168a11d67ec7cc4695ec35592cee0d4f7ac8267dd09543dad","kind":1,"pubkey":"787338757fc25d65cd929394d5e7713cf43638e8d259e8dcf5c73b834eb851f2","sig":"c2c4a6f76aab7f25236acffabd15e9c9cd9874d1f410355ab7bca254086bd3a838fe68cf6e48870e993e5a69eef20a4d93d9dd6e08af45f29c0ac7aa141ddc7a","tags":[["t","nostrdesign"],["t","nostrdesign"],["r","https://nostrdesign.org/"]]}'

In [50]:
json.loads(event.as_json())

{'content': "Thank you for building, Josi. We're happy to support you for the long term \U0001fae1\n\nhttps://opensats.org/blog/josi-baker-receives-opensats-lts-grant",
 'created_at': 1692219512,
 'id': 'bb11666d51331ed440b8f573de42b798e0de67a1cea7d5ea5d9959c3b9bbfd51',
 'kind': 1,
 'pubkey': '787338757fc25d65cd929394d5e7713cf43638e8d259e8dcf5c73b834eb851f2',
 'sig': 'bd74dfe3fdafad7205882c5cf9c61c68d0609ecdd21d232718cad26503b814a5588bc596c3de65c95667085631ea510b7045f509a94026c68d2d2df327f821d8',
 'tags': []}